In [34]:
import picamera
import time
from IPython.display import Image
import math
import serial
import re
import cv2 as cv

In [35]:
IMAGES_PATH = "images/"
utils = "Utils/"

face_cascade = cv.CascadeClassifier(utils + 'bottle.xml')

In [40]:
''''''
''' TCP COMMUNICATION '''
''''''

def enable_tcp_com(BUFFER_SIZE):
    TCP_IP = '192.168.0.21' 
    TCP_PORT = 5005
    while(1):
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect((TCP_IP, TCP_PORT))
            s.setblocking(0)
        except:
            continue
        else:
            print("TCP com enabled!")
            break

    return s

def send_message(s, msg):
    s.send((msg).encode())
    
def check_data_received(s, msg, BUFFER_SIZE):
    ready = select.select([s], [], [], 0.2)
    print(ready)
    if ready[0]:
        data = s.recv(BUFFER_SIZE)
        print(data.decode())
        return data.decode()[-9:] == msg
    else :
        print("Data was not received")
        return None

def close_connection(s):
    s.close()

''''''
''' Robot position '''
''''''


def open_ser():
    ser = serial.Serial(
        port='../../../dev/ttyS0', #Replace ttyS0 with ttyAM0 for Pi1,Pi2,Pi0
        baudrate = 115200,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.EIGHTBITS,
        timeout=1
    )
    return ser

def get_robot_pos_angle(ser):
    buffer = ''
    timeout = time.time() + 5 #5s timeout
    while True:
        # last_received = ser.readline()
        buffer += str(ser.read(ser.inWaiting()))
        if '\\n' in buffer:
            last_received = buffer.split('\\n')[-2]
            pos_angle = re.findall('\d+\.\d+', last_received )
            if len(pos_angle) == 3:
                return pos_angle[0], pos_angle[1], pos_angle[2]
        if time.time() > timeout:
            break


''''''
''' Bottle position '''
''''''

def calcul_bottle_pos(xr,yr,ar,dist_rb,a_rb):
    ar = ar - 90
    ar = ar % 360
    
    ab = ar + a_rb
            
    rel_xb = dist_rb * math.sin(math.radians(ab))
    rel_yb = dist_rb * math.cos(math.radians(ab))
    
    real_xb = xr + rel_xb
    real_yb = yr + rel_yb
    return real_xb,real_yb


def calcul_bottle_dist_angle(x,y,w,h):
    # TODO
    return 200, 20

In [ ]:
camera = picamera.PiCamera()
camera.rotation = 180
camera.capture(IMAGES_PATH + "cam.bmp")

In [ ]:
img = cv.imread(IMAGES_PATH + 'bottle.bmp')

In [ ]:
# resize to increase speed detection
rows,cols = img.shape[:2]   
img = cv.resize(img,(round(cols*0.7),round(rows*0.7)))

In [ ]:
start_time = datetime.datetime.now()
faces = face_cascade.detectMultiScale(img)
print("time: " + str(datetime.datetime.now() - start_time))   
for (x,y,w,h) in faces:
    cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
cv.imwrite(IMAGES_PATH + "cam_haar.png", img)
Image(filename = IMAGES_PATH + 'cam_haar.png')

In [24]:
ser = open_ser()

In [41]:
start_time = datetime.datetime.now()
x, y, angle = get_robot_pos_angle(ser)
end_time = datetime.datetime.now()
print(str(end_time - start_time))

0:00:00.002596
